# Week 10 -- MapReduce using PySpark
## Load the data

In [31]:
import pandas as pd

df = pd.read_csv("small_twitter_data.csv", encoding='latin1') # dataframe has no headers :(

cols = ['sentiment','id','date','query_string','user','text']
df = pd.read_csv("small_twitter_data.csv", encoding='latin1', names=cols) # this version has headers :)

# we just want the tweet column:
data = df['text'].tolist()
print(data[0])



@stellargirl I loooooooovvvvvveee my Kindle2. Not that the DX is cool, but the 2 is fantastic in its own right.


## Word counter

In [33]:
from collections import Counter

# counts how many times each word appears in the dataset
def count_words(data):
    counter = Counter()
    for tweet in data:
        words = tweet.split()
        counter.update(words)
    return counter

counter = count_words(data)
print(counter.most_common(5))

[('the', 194), ('to', 142), ('a', 123), ('I', 120), ('is', 104)]


## Word counter with cleaner data

In [37]:
import sys
sys.path.append('/content')
from util import *

def count_words(data):
    counter = Counter()
    for tweet in data:
        words = tweet.split()
        cleaned = map(clean_word, words)
        filtered = filter(word_not_in_stopwords, cleaned)
        counter.update(filtered)
    return counter

counter = count_words(data)
print(counter.most_common(5))

[('time', 45), ('night', 37), ('new', 36), ('good', 34), ('warner', 32)]


## PySpark Approach

In [39]:
cols = ['sentiment', 'id', 'date', 'query_string', 'user', 'text']
df = pd.read_csv("small_twitter_data.csv", encoding = 'latin1', names = cols)
data = df['text'].tolist()


In [41]:
from pyspark import SparkContext, SparkConf

conf = SparkConf().setAppName('MapReduce').setMaster('local')
sparkContext = SparkContext.getOrCreate(conf=conf) #SparkContext(conf = conf)

def mapper(text):
    words = text.split()
    cleaned = map(clean_word, words)
    filtered = filter(word_not_in_stopwords, cleaned)
    return Counter(filtered)

def reducer(counter1, counter2):
    counter1.update(counter2)
    return counter1

distributed_data = sparkContext.parallelize(data, 10) # 10 chunks
dist_data_mapped = distributed_data.map(mapper)
dist_data_reduced = dist_data_mapped.reduce(reducer)
dist_data_reduced.most_common(5)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0) (LAPTOP-BPQQP4BN executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:67)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:367)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:331)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:93)
	at org.apache.s